## Prompt Lab Challenge Exercises Notebook

Welcome to the second prompt lab in the bootcamp series, you should have completed lab 1 and the exercises follow on from those. If you completed all the exercises in Lab 1 you should find most of the exercises here straightforward.

This notebook is a template with all the exercises and indications of what the output should look like if you successfully engineer the prompt.

Before you start you should have a Python environment with the necessary libraries installed as indicated in the intro lab, you will also need a .env file with your watsonx.ai KEY to load your credentials.

It should take you about 30-45 min to walk through the exercises self paced

Good luck and make sure you compare your answers with the model solutions


1. Import modules
    - note we are using the internal watsonx.ai for these exercises so you need to be on VPN/IBM Network
    - the code should run the same if you are using ibm-generative-ai url instead of the watsonx.ai url - just update my_api_endpoint in #2 below.


In [8]:
import os
from dotenv import load_dotenv
from genai.schemas import GenerateParams
from genai.model import Credentials, Model

2. Load credentials for watsonx.ai (note refer to lab explaining how to do this if necessary)
    - you should have a .env file with your API key, eg GENAI_KEY=xxx
    - you should have a .env api endpoint , eg GENAI_API=https://workbench-api.res.ibm.com/v1/

In [9]:
#config watsonx.ai environment
load_dotenv()
api_key = os.getenv("GENAI_KEY", None)
api_endpoint = os.getenv("GENAI_API", None)
print(api_endpoint)
if api_key is None or api_endpoint is None:
    print("ERROR: Ensure you copied the .env file that you created earlier into the same directory as this notebook")
else:
    creds = Credentials(api_key=api_key, api_endpoint=api_endpoint)

https://workbench-api.res.ibm.com/v1/


#### Helper function for genai. 
Take a look at its tunable parameters. You will need to change some of these parameters to fine-tune your response to the following questions.

In [15]:
mt_model = 'bigscience/mt0-xxl'
flanul_model = 'google/flan-ul2'

In [23]:
def send_to_watsonxai(prompts,
                    decoding_method="greedy",
                    max_new_tokens=100,
                    min_new_tokens=30,
                    temperature=1.0,
                    repetition_penalty=2.0
                    ):
    '''
   helper function for sending prompts and params to watsonx.ai
    
    Args:  
        prompts:list list of text prompts
        decoding:str watsonx.ai parameter "sample" or "greedy"
        max_new_tokens:int watsonx.ai parameter for max new tokens/response returned
        min_new_tokens:int watsonx.ai parameter for min new tokens/response returned
        temperature:float watsonx.ai parameter for temperature (range 0>2)
        repetition_penalty:float watsonx.ai parameter to penalize repetition

    Returns: None
        prints response
    '''


    # Instantiate parameters for text generation
    params = GenerateParams(decoding_method=decoding_method, 
                            min_new_tokens=min_new_tokens,
                            max_new_tokens=max_new_tokens,
                            random_seed=42,
                            temperature=temperature,
                            repetition_penalty=repetition_penalty)

    # Instantiate a model proxy object to send your requests
    model = Model(flanul_model, params=params, credentials=creds)

    for response in model.generate(prompts):
        print(response.generated_text)

#### Product Review for Questions  1-5

In [5]:

lamp_review = """Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!"""

#### Q0) EXAMPLE QUESTION: write a prompt to identify and return the issues that the customer had with their purchased product.
Target response = - the string to our lamp broke during the transit

In the following cell, we have constructed a prompt to be input into the `send_to_watsonxai` function:
- First we tell the model the goal and how we will denote the input
- We write the `lamp_review` into the prompt using [fstring](https://realpython.com/python-f-strings/#simple-syntax) formatting. 
- We tell the model how we expect its response to be formatted.
- We have changed the`min_new_tokens` and `max_new_tokens` parameters in the `send_to_watsonxai` function to restrict the model's response to the desired length.
  
Keep in mind that we aren't using any multi-shot learning techniques in this example. Your responses to the questions in this lab may need to include different prompt-engineering techniques.

In [6]:
#Q0 Code - enter prompt and parameters in this cell
prompt = f"""Identify the issues that the customer had with the product in the following review denoted by triple backticks.

```{lamp_review}```

Make your response as short as possible. \
The response must not include anything positive about the product. \
Write the response as a list of one reason denoted with '-' characters.\
"""
send_to_watsonxai(prompts=[prompt],
                  min_new_tokens=10,
                  max_new_tokens=35,
                  repetition_penalty=2,
                  temperature=1)

- the string to our lamp broke during the transit


#### Q1) write a prompt to return the sentiment of the review
Target sentiment = positive

In [7]:
#Q1 Code - enter prompt and function parameters in this cell
prompt = f"identify the sentiment in the following product review: {lamp_review}"
send_to_watsonxai(prompts=[prompt],
                  min_new_tokens= 1,
                  max_new_tokens= 2)

positive


#### Q2) extract the emotions the reviewer expressed, return answer as a comma separated list
Target emotions = satisfied, happy, cared for, great company, product!

In [9]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""

send_to_watsonxai(prompts=[prompt],min_new_tokens=10,max_new_tokens=30)

satisfied, happy, cared for, great company


#### Q3) Is the reviewer expressing anger, answer “yes” or “no” – test with your own example including anger to ensure it works in both cases.
Target answer = no

In [10]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''

"""

send_to_watsonxai(prompts=[prompt],max_new_tokens=2,min_new_tokens=1)

no


#### Q4) Extract the item purchased and the company name, return as JSON format
Target answer = Item[lamp], Brand[Lumina]

In [11]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""

send_to_watsonxai(prompts=[prompt],max_new_tokens=30,min_new_tokens=10)

Item[lamp], Brand[Lumina]


#### Q5) Can you combine 1-5 in a single prompt and return JSON with: Sentiment (negative or positive), Anger (yes/no), Product, Company
Target answer = Sentiment[positive], Anger[false], Item[lamp], Brand[Lumina]

In [12]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""

send_to_watsonxai(prompts=[prompt],max_new_tokens=50,min_new_tokens=20)


Sentiment[positive], Anger[false], Item[lamp], Brand[Lumina]


#### Q6) summarize the following product review
Example summary = My daughter loves it!  It's soft and  super cute, and its face has a friendly look. It's  a bit small for what I paid though.

In [13]:
review = """Got this panda plush toy for my daughter's birthday, \
who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's \ 
a bit small for what I paid though. I think there \ 
might be other options that are bigger for the \ 
same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it to her."""

<>:1: DeprecationWarning: invalid escape sequence '\ '
<>:1: DeprecationWarning: invalid escape sequence '\ '
/var/folders/1k/qhhfnfrs2632tlslj6sb7sqh0000gn/T/ipykernel_64279/3719332633.py:1: DeprecationWarning: invalid escape sequence '\ '
  review = """Got this panda plush toy for my daughter's birthday, \


In [14]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words. 

Review: ```{review}```
"""

send_to_watsonxai(prompts=[prompt],max_new_tokens=50,min_new_tokens=10)

My daughter loves it! but it's a bit small for what I paid


#### Q7) Summarize the same product review from the perspective of the shipping department
Example summary = It arrived a day earlier than expected, so I got to play with it myself before I gave it  to her. 

In [15]:
#concise wrt feedback shipping
prompt = f""" 

read the review below, highlighted between triple backticks, and tell me about the delivery of the product

Review: ```{review}```
"""

send_to_watsonxai(prompts=[prompt],max_new_tokens=50, min_new_tokens=10)

It arrived a day earlier than expected,  so I got to play with it myself before I gave it to her.


#### Q8) Summarize the review from the perspective of pricing and value
Example summary = It's a bit small for what I paid though. I think there might be other options that are bigger for the same price

In [16]:
#feedback pricing works - concise
prompt = f"""
You should read the review below, delimited by triple 
backticks, summarize any aspects \
that are relevant to the price and perceived value.  

Review: ```{review}```
"""

send_to_watsonxai(prompts=[prompt],min_new_tokens=10,max_new_tokens=50)


My daughter loves it, but it's a bit small for what I paid


#### Q9)	PII removal. Given the following email, write a prompt to remove the PII (eg names, emails etc) (Hint: you may need to use 1-2 shot technique, consider using ChatGPT to generate your shots)

Desired response:
```
Hi [NAME],

I'm writing to you because I noticed you recently purchased a new car. 
I'm a salesperson at a local dealership (Cheap Dealz), and I wanted to let you know that we have a great deal on a new car. If you're interested, please let me know.

Thanks,
[NAME]
Phone: [PHONE]
Email: [EMAIL]
```

In [12]:
email = """
Hi John,\

I'm writing to you because I noticed you recently purchased a new car. I'm a salesperson\
at a local dealership (Cheap Dealz), and I wanted to let you know that we have a great deal on a new\
car. If you're interested, please let me know.\

Thanks,\

Jimmy Smith\

Phone: 410-805-2345\
Email: jimmysmith@cheapdealz.com\
"""

ex_1 = """
Hi Xxxxx,\

I'm writing to you because I noticed you recently purchased a new car. I'm a salesperson\
at a local dealership (Cheap Dealz), and I wanted to let you know that we have a great deal on a new\
car. If you're interested, please let me know.\

Thanks,\

Sam Jones\

Phone: 411-805-7345\
Email: samjones@cheapercars.com\
"""

ans_1 = """
Hi [NAME],\

I'm writing to you because I noticed you recently purchased a new car. I'm a salesperson\
at a local dealership (Cheap Dealz), and I wanted to let you know that we have a great deal on a new\
car. If you're interested, please let me know.\

Thanks,\

[NAME]\

Phone: [PHONE]\
Email: [EMAIL]\


"""

In [18]:
prompt = f"""Read the following sales email. Remove any personally identifiable information (PII),and replace it with the appropriate placeholder. For example, replace the name "Dave Doe" with "[NAME]". 

{ex_1}

{ans_1}

{email}
"""

send_to_watsonxai(prompts=[prompt], min_new_tokens=20, max_new_tokens=100)

Hi [NAME], I'm writing to you because I noticed you recently purchased a new car. I'm a salespersonat a local dealership (Cheap Dealz), and I wanted to let you know that we have a great deal on a newcar. If you're interested, please let me know. Thanks, [NAME] Phone: [PHONE]Email: [EMAIL]


#### Q10) Mathematical Inference: A patients a1c level determines their diabetes status, the rules are as follows:

 - less than 5.7 no diabetes
 - between 5.7 and 6.5 pre-diabetes
 - greater than 6.5 diabetic.

Write a prompt to return just the diabetes status from the following 3 test cases:

1)	The patients a1c is 5.5 which is good considering his other risk factors.
2)	From the last lab report I noted the A1c is 6.4 so we need to put her on Ozempic.
3)	She mentioned her A1c is 8 according to her blood work about 3 years ago.

Bonus 1: How could you improve the inference given the other information in the sentences?

Bonus 2: how would you approach extracting the diabetes status based on patient notes without A1C values and what would you need to watch out for? (hint: maybe they are talking about family history of disease or other complications)


In [7]:
#Q1 ENTER YOUR MODEL PARAMS HERE - MAKE SURE IT WORKS WITH ALL 3 EXAMPLES ABOVE
prompt = f"""Diabetes status is indicated by a patients a1c level. the following a1c ranges determine the diabetes status.
First, read the a1c level from the text delimited by triple back-ticks.

Next, if the a1c level is less than 5.7 then output diabetes status: no-diabetes
if a1c level is greater than 6.5: then output diabetes status: diabetes
if the a1c is between 5.8 and 6.4, then output diabetes status: pre-diabetes

'''She mentioned her A1c is 8 according to her blood work about 3 years ago.'''

what is the diabetes status:"""

send_to_watsonxai(prompts=[prompt],max_new_tokens=10,min_new_tokens=1) 

diabetes


After you feel satisfied with your prompt, try changing single words in the patient test cases to see how your model responds. That is, in test case 1 change the word '`good`' to '`bad`' to see how your model may be interpreting the surrounding context in evaluating its response.

### alternate solution to Q9 PII removal

In [25]:
email = """Hi Bob, I'm writing to you because I noticed you recently purchased a new car. I'm a salesperson at a local dealership (Cheap Dealz), and I wanted to let you know that we have a great deal on a newcar. If you're interested, please let me know. Thanks, Jimmy Smith Phone: 410-805-2345Email: jimmysmith@cheapdealz"""

In [29]:
prompt_ = f"""Read the following email and identify any names, email addresses, or other personal information. Then replace any personal information found with a placeholder. 

greetings ken,

Wanted to let you know it was good seeing you at Radio City Music Hall. Hope we can do something else fun soon!

James Grime

Phone: 212-855-2213
Email: jgrimmy@gmail.com


greetings [NAME],

Wanted to let you know it was good seeing you at [LOCATION]. Hope we can do something else fun soon!

[NAME]

Phone: [PHONE]
Email: [EMAIL]
 
{email}

"""

In [30]:
send_to_watsonxai(prompts=[prompt], min_new_tokens=20, max_new_tokens=100)

Hi [NAME], I'm writing to you because I noticed you recently purchased a new car. I'm a salespersonat a local dealership (Cheap Dealz), and I wanted to let you know that we have a great deal on a newcar. If you're interested, please let me know. Thanks, [NAME] Phone: [PHONE]Email: [EMAIL]
